# Section 5: Model Registry & 本番活用

## 🎯 このセクションの目標
- Model Registryの概念と利点を理解する
- 学習済みモデルをModel Registryに登録する
- **バージョン管理（v1 → v2）** を体験する
- 登録したモデルを使って本番推論を実行する

## 💡 Model Registryとは？

機械学習モデルを一元管理するためのシステムです。

**主な利点:**
- 📦 **モデルのバージョン管理**（v1, v2, ...）
- 📝 メタデータ（メトリクス、説明）の管理
- 🔐 アクセス制御とガバナンス
- 🚀 SQL/Pythonからのシームレスな推論
- 🔄 **デフォルトバージョン**の設定

---
## 5.1 環境設定

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from snowflake.snowpark.context import get_active_session
from snowflake.snowpark import functions as F

# Snowpark ML モデリング
from snowflake.ml.modeling.xgboost import XGBClassifier

# Model Registry
from snowflake.ml.registry import Registry

# 評価メトリクス（sklearn使用）
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# セッションの取得
session = get_active_session()

# 環境設定
session.use_database("MLOPS_HOL_DB")
session.use_warehouse("MLOPS_HOL_SQL_WH")

print("✅ ライブラリのインポート完了")

---
## 5.2 学習データの準備

Section 4で選択した最良のパラメータでモデルを学習します。

In [ ]:
# 学習データの読み込み
training_data = session.table("MLOPS_HOL_DB.FEATURE_STORE.TRAINING_DATASET_V1")

# 特徴量の定義（Section 2-4と同じ）
FEATURE_COLS = [
    "DAYS_SINCE_LAST_ORDER",
    "TOTAL_ORDER_COUNT",
    "ORDER_COUNT_2024",
    "TOTAL_ORDER_AMOUNT",
    "AVG_ORDER_AMOUNT",
    "TOTAL_AMOUNT_2024",
    "RETURN_RATE"
]
LABEL_COL = "IS_CHURNED"
ID_COL = "CUSTOMER_ID"

# 訓練・テストデータの分割
model_data = training_data.select([ID_COL] + FEATURE_COLS + [LABEL_COL])
train_df, test_df = model_data.random_split([0.8, 0.2], seed=42)

print(f"訓練データ: {train_df.count():,} 件")
print(f"テストデータ: {test_df.count():,} 件")

In [ ]:
# Section 4で選んだ最良パラメータでモデルを学習
best_params = {
    "max_depth": 5,
    "learning_rate": 0.1,
    "n_estimators": 150,
    "scale_pos_weight": 3
}

print("🔄 モデルを学習中...")

best_model = XGBClassifier(
    input_cols=FEATURE_COLS,
    label_cols=LABEL_COL,
    output_cols="PREDICTED_CHURN",
    **best_params,
    random_state=42
)

best_model.fit(train_df)

# 評価
predictions = best_model.predict(test_df)
pred_pd = predictions.to_pandas()
y_true = pred_pd[LABEL_COL]
y_pred = pred_pd["PREDICTED_CHURN"]

metrics = {
    "accuracy": accuracy_score(y_true, y_pred),
    "precision": precision_score(y_true, y_pred),
    "recall": recall_score(y_true, y_pred),
    "f1_score": f1_score(y_true, y_pred)
}

print("✅ モデル学習完了")
print(f"   Accuracy:  {metrics['accuracy']:.4f}")
print(f"   Precision: {metrics['precision']:.4f}")
print(f"   Recall:    {metrics['recall']:.4f}")
print(f"   F1 Score:  {metrics['f1_score']:.4f}")

---
## 5.3 Model Registryへの登録

In [ ]:
# Model Registryのインスタンス作成
registry = Registry(
    session=session,
    database_name="MLOPS_HOL_DB",
    schema_name="MODEL_REGISTRY"
)

print("✅ Model Registry接続完了")

In [ ]:
# 既存モデルのクリーンアップ（再実行用）
MODEL_NAME = "CUSTOMER_CHURN_PREDICTOR"

try:
    existing_model = registry.get_model(MODEL_NAME)
    existing_model.delete_version("v1")
    print("Deleted existing v1")
except:
    pass

try:
    existing_model = registry.get_model(MODEL_NAME)
    existing_model.delete_version("v2")
    print("Deleted existing v2")
except:
    pass

In [ ]:
# モデルの登録
VERSION_NAME = "v1"

print(f"🔄 モデルを登録中: {MODEL_NAME} ({VERSION_NAME})")

model_ref = registry.log_model(
    model=best_model,
    model_name=MODEL_NAME,
    version_name=VERSION_NAME,
    metrics=metrics,
    comment="チャーン予測モデル v1 - RFM特徴量ベース"
)

print("✅ モデル登録完了！")
print(f"   モデル名: {MODEL_NAME}")
print(f"   バージョン: {VERSION_NAME}")

In [ ]:
# 登録されたモデルの確認
print("=== 登録済みモデル一覧 ===")
registry.show_models()

In [ ]:
# モデルのバージョン情報を確認
print(f"=== {MODEL_NAME} のバージョン情報 ===")
model = registry.get_model(MODEL_NAME)
model.show_versions()

---
## 5.4 バージョン管理の体験（v1 → v2）

Feature Storeと同様に、Model Registryでも**バージョン管理**が可能です。

### v2 での改善点
- `n_estimators` を 150 → 200 に増加
- `max_depth` を 5 → 6 に増加
- より複雑なパターンを学習

In [ ]:
# v2用の改良パラメータ
v2_params = {
    "max_depth": 6,           # 5 → 6 に増加
    "learning_rate": 0.1,
    "n_estimators": 200,      # 150 → 200 に増加
    "scale_pos_weight": 3
}

print("🔄 v2モデルを学習中...")
print(f"   改善点: max_depth={v2_params['max_depth']}, n_estimators={v2_params['n_estimators']}")

v2_model = XGBClassifier(
    input_cols=FEATURE_COLS,
    label_cols=LABEL_COL,
    output_cols="PREDICTED_CHURN",
    **v2_params,
    random_state=42
)

v2_model.fit(train_df)

# v2モデルの評価
v2_predictions = v2_model.predict(test_df)
v2_pred_pd = v2_predictions.to_pandas()
v2_y_true = v2_pred_pd[LABEL_COL]
v2_y_pred = v2_pred_pd["PREDICTED_CHURN"]

v2_metrics = {
    "accuracy": accuracy_score(v2_y_true, v2_y_pred),
    "precision": precision_score(v2_y_true, v2_y_pred),
    "recall": recall_score(v2_y_true, v2_y_pred),
    "f1_score": f1_score(v2_y_true, v2_y_pred)
}

print("✅ v2モデル学習完了")
print(f"   Accuracy:  {v2_metrics['accuracy']:.4f}")
print(f"   Precision: {v2_metrics['precision']:.4f}")
print(f"   Recall:    {v2_metrics['recall']:.4f}")
print(f"   F1 Score:  {v2_metrics['f1_score']:.4f}")

In [ ]:
# v2モデルをModel Registryに登録
VERSION_V2 = "v2"

print(f"🔄 v2モデルを登録中: {MODEL_NAME} ({VERSION_V2})")

model_ref_v2 = registry.log_model(
    model=v2_model,
    model_name=MODEL_NAME,
    version_name=VERSION_V2,
    metrics=v2_metrics,
    comment="チャーン予測モデル v2 - パラメータ改善版"
)

print("✅ v2モデル登録完了！")
print(f"   モデル名: {MODEL_NAME}")
print(f"   バージョン: {VERSION_V2}")

In [ ]:
# バージョン一覧の確認
print(f"=== {MODEL_NAME} のバージョン一覧 ===")
model = registry.get_model(MODEL_NAME)
model.show_versions()

In [ ]:
# v1 vs v2 の比較
print("=" * 70)
print("📊 モデルバージョン比較")
print("=" * 70)
print(f"{'メトリクス':<15} {'v1':>12} {'v2':>12} {'差分':>12}")
print("-" * 70)

for metric_name in ['accuracy', 'precision', 'recall', 'f1_score']:
    v1_val = metrics[metric_name]
    v2_val = v2_metrics[metric_name]
    diff = v2_val - v1_val
    diff_str = f"{diff:+.4f}"
    print(f"{metric_name:<15} {v1_val:>12.4f} {v2_val:>12.4f} {diff_str:>12}")

print("=" * 70)

if v2_metrics['f1_score'] > metrics['f1_score']:
    print("✅ v2はF1スコアが改善しています！")
elif v2_metrics['f1_score'] < metrics['f1_score']:
    print("⚠️ v1の方がF1スコアが高いです")
else:
    print("➡️ F1スコアは同じです")

### 💡 バージョン管理のポイント

| 操作 | 説明 |
|------|------|
| `registry.log_model(..., version_name="v2")` | 新しいバージョンとして登録 |
| `model.show_versions()` | 全バージョン一覧を表示 |
| `model.version("v1")` | 特定バージョンを取得 |
| `model.default` | デフォルトバージョンを取得 |

**実務での活用例:**
- **A/Bテスト**: v1とv2で異なる顧客グループに適用し、効果を比較
- **ロールバック**: v2に問題があればv1に戻す
- **監査**: 過去のバージョンで予測された結果の再現

---
## 5.5 登録モデルを使った推論

Model Registryに登録されたモデル（v2）を使って、全顧客に対してチャーンスコアを計算します。

In [ ]:
# v2モデルでテストデータを推論
model_version = model.version(VERSION_V2)  # v2を使用

# 推論実行
inference_result = model_version.run(
    test_df.select([ID_COL] + FEATURE_COLS),
    function_name="predict"
)

print(f"✅ 推論完了（{MODEL_NAME} {VERSION_V2}を使用）")
inference_result.show(10)

---
## 5.6 チャーンリスク顧客リストの作成

In [ ]:
# チャーンリスク高の顧客を抽出
high_risk_customers = inference_result.filter(
    F.col("PREDICTED_CHURN") == 1
)

print(f"🎯 チャーンリスク高の顧客数: {high_risk_customers.count():,}")

In [ ]:
# 顧客情報と結合
customer_info = session.table("MLOPS_HOL_DB.PREP_DATA.CUSTOMERS")

target_list = high_risk_customers.join(
    customer_info,
    on="CUSTOMER_ID",
    how="inner"
).select(
    F.col("CUSTOMER_ID"),
    F.col("SEGMENT"),
    F.col("REGION"),
    F.col("PREDICTED_CHURN").alias("CHURN_RISK")
)

print("=== チャーンリスク高の顧客リスト（サンプル） ===")
target_list.show(10)

In [ ]:
# ターゲットリストをテーブルに保存
target_list.write.save_as_table(
    "MLOPS_HOL_DB.ANALYTICS.CHURN_RISK_CUSTOMERS",
    mode="overwrite"
)

print("✅ チャーンリスク顧客リストを保存しました")
print("   テーブル名: MLOPS_HOL_DB.ANALYTICS.CHURN_RISK_CUSTOMERS")

---
## 5.7 ビジネスアウトカムの確認

In [ ]:
# サマリーレポート
total_test_customers = test_df.count()
high_risk_count = high_risk_customers.count()
churn_rate = high_risk_count / total_test_customers * 100

print("=" * 60)
print("🎯 ビジネスアウトカム サマリー")
print("=" * 60)
print(f"  テスト顧客数:            {total_test_customers:,} 人")
print(f"  チャーンリスク高顧客数:  {high_risk_count:,} 人")
print(f"  予測チャーン率:          {churn_rate:.1f}%")
print("-" * 60)
print(f"  📧 リテンション施策対象: {high_risk_count:,} 人")
print("=" * 60)

In [ ]:
# セグメント別のチャーンリスク分布
print("=== セグメント別チャーンリスク分布 ===")
target_list.group_by("SEGMENT") \
    .count() \
    .order_by(F.col("COUNT").desc()) \
    .show()

In [ ]:
# 地域別のチャーンリスク分布
print("=== 地域別チャーンリスク分布 ===")
target_list.group_by("REGION") \
    .count() \
    .order_by(F.col("COUNT").desc()) \
    .show()

---
## 5.8 SQLからのモデル呼び出し

Model Registryに登録されたモデルは、**SQLから直接呼び出す**ことができます。

In [ ]:
print("=== SQLでのモデル呼び出し例 ===")
sql_example = """
-- 新しい顧客データに対してチャーン予測を実行
SELECT 
    CUSTOMER_ID,
    MLOPS_HOL_DB.MODEL_REGISTRY.CUSTOMER_CHURN_PREDICTOR!PREDICT(
        DAYS_SINCE_LAST_ORDER,
        TOTAL_ORDER_COUNT,
        ORDER_COUNT_2024,
        TOTAL_ORDER_AMOUNT,
        AVG_ORDER_AMOUNT,
        TOTAL_AMOUNT_2024,
        RETURN_RATE
    ) AS CHURN_PREDICTION
FROM CUSTOMER_FEATURES_VIEW
WHERE CHURN_PREDICTION = 1;
"""
print(sql_example)

---
## ✅ Section 5 完了！ハンズオン終了！ 🎉

### 学んだこと（全セクション）

| セクション | 内容 |
|-----------|------|
| Section 1 | データ探索、**チャーンの定義・ラベル作成** |
| Section 2 | Feature Store（Entity, FeatureView, バージョン管理） |
| Section 3 | XGBoostでのモデル学習、CV、SHAP |
| Section 4 | Experiment Tracking（実験管理、モデル比較） |
| Section 5 | Model Registry（モデル登録、バージョン管理、本番推論） |

### 構築したもの

- ✅ **CHURN_LABELS**: 定義・作成したチャーンラベル
- ✅ **Feature Store**: 7個の特徴量（RFM + 行動）、v1→v2バージョン管理
- ✅ **実験管理**: 3つのモデルを比較し、最適なものを選択
- ✅ **Model Registry**: 本番モデルを登録、v1→v2バージョン管理
- ✅ **チャーンリスク顧客リスト**: リテンション施策の対象リスト

### バージョン管理の比較

| 機能 | Feature Store | Model Registry |
|------|--------------|----------------|
| バージョン | v1, v2, ... | v1, v2, ... |
| 改善方法 | 特徴量追加・修正 | パラメータ・アルゴリズム変更 |
| 比較 | 特徴量の差分確認 | メトリクスの比較 |
| ロールバック | 過去バージョンを参照 | 過去バージョンで推論 |

### 次のステップ（発展編）

1. **ハイパーパラメータ最適化**: `Tuner` を使った自動チューニング
2. **モデルモニタリング**: データドリフトの検出
3. **リアルタイム推論**: Snowpark Container Services
4. **CI/CD連携**: GitHub Actions + Snowflake CLI

---
## 📚 参考リンク

- [Snowflake ML Overview](https://docs.snowflake.com/en/developer-guide/snowflake-ml/overview)
- [Snowflake Feature Store](https://docs.snowflake.com/en/developer-guide/snowflake-ml/feature-store/overview)
- [Snowflake Model Registry](https://docs.snowflake.com/en/developer-guide/snowflake-ml/model-registry/overview)
- [Snowflake ML Experiments](https://docs.snowflake.com/en/developer-guide/snowflake-ml/experiments)
- [Snowpark ML Modeling](https://docs.snowflake.com/en/developer-guide/snowflake-ml/modeling)